**Essentials**

1. Write one function for each of the following methods: the bisection method, the secant method and the Newtond's method. They must find the root of a given function with a given precision. The code should also return the number of iterations, the time that the method needed to achieve that precision and the values of the approximations to the root at each iteration. 

**Note:** Start coding just one of them (Secant or Newton's, as bisection was already coded for you in the class notebook) and use it to solve the exercises. Modifying the code to use the other should be easy. do it at the end of the practice if you have time left, or at home as homework.

2. Determine the point of intersection of the curves given by $y=x^3-2x+1$ and $y=x^2$.
3. Find the rectangle of maximum area if its vertices are $(0,0)$, $(x,0)$,$(x,\cos(x))$ and $(0,\cos(x))$ for $x\in[0,2\pi]$.
4. In Newton's (or secant) method, we progress in each step from a given point $x$ to a new point $x-h$, where $h = f(x)/f'(x)$ (or the equivalent for the secant method). A refinement that is easily programmed is this: If $|f (x - h)|$ is not smaller than $|f (x)|$, then reject this value of $h$ and use $h/2$ instead. Test this refinement (for example, use $f(x)=\arctan(x)$ with $x_0=2$).

**Solving simultaneous nonlinear equations.**

Newton's method can be adapted to solve systems of nonlinear equations "easily". Check the notes in Aula global.

5. Use the Newton's method to solve the find a zero of the following equations:

$$\begin{array}{l} x^2+y^2-25=0\\  x^2-y-2=0 \end{array}$$

using a suitable starting point.

**Once you have done the previous exercises and coded the three methods:**

6. Solve exercises 2, 3 with the three methods and compare the results in terms of time and iterations for different precision tolerances. Use $\epsilon=1e-1, 1e-2, 1e-5, 1e-10$.

# Solution 

In [7]:
import sys 
from typing import Callable

import numpy as np

## Exercise 1

### Bisection method

In [8]:
def bisection(f: 'Callable[float]', a: float, b: float, err: float, Nmax: int = 100_000) -> float:
    r"""Computes Bisection method to find roots f(x)=0.

    If there are no roots in the interval :math:`[a, b]`, the method will throw an exception.
    This is checked using bolzano's theorem (If :math:`f(a)*f(b) >= 0`).

    Args:
        f (Callable[float]): Function of which we want to find roots :math:`f(x)=0`.
        a (float): Lower bound of the interval.
        b (float): Upper bound of the interval.
        err (float): Tolerance of the result. It assures that the root is in :math:`[x-err, x+err]`. #TODO: Is this the interval?
        Nmax (int): Maximum number of iterations. Defaults to 100_000.

    Raises:
        ValueError: If, according to Bolzano's theorem, there cannot be roots in :math:`[a, b]`.
        ValueError: If the method, being at least one root in :math:`[a, b]`, fails to to compute the root.

    Returns:
        float: Root x such as f(x)=0 with a tolerance err.

    Examples:
        >>> f = lambda x: (x**2 - 1)
        >>> bisection(f, -0.5, 2, 1e-10)
        2.9103830456733704e-11
        >>> bisection(f, -0.5, 2, 1e-10, 100)
        ValueError: Could not find a root in the interval [-0.5, 2] with tolerance 1e-10 in 5 iterations.
        >>> bisection(f, 5, 20, 1e-7)
        ValueError: f(a)*f(b)=9576 <0.   No roots in this interval.
    """
    if f(a)*f(b) >= 0:
        raise ValueError(f'f(a)*f(b) = {f(a)*f(b)} <0. \t No roots in this interval.')

    N = int(min(Nmax, np.ceil(np.log((b-a)/err) / np.log(2) - 1)))  # What is this?
    a_n = a
    b_n = b
    m_n = (a_n + b_n)/2
    f_m_n = f(m_n)
    for _ in range(1, N+1):

        if f(a_n)*f_m_n < 0:
            b_n = m_n

        elif f(b_n)*f_m_n < 0:
            a_n = m_n

        m_n = (a_n + b_n)/2
        f_m_n = f(m_n)

        if abs(f_m_n) <= err:
            return m_n

    raise ValueError(f'Could not find a root in the interval [{a}, {b}] with tolerance {err} in {N} iterations.')


### Secant method

In [9]:
def secant(f: 'Callable[float]', a: float, b: float, err: float, Nmax: int = 100_000, x0: float = None) -> float:
    r"""Computes Secant method to find roots :math:`f(x)=0`.

    If there are no roots in the interval :math:`[a, b]`, the method will throw an exception.
    This is checked using bolzano's theorem (If :math:`f(a)*f(b) >= 0`).

    To computes the first iteration, it computes the previous value as :math: `a-1`

    Args:
        f (Callable[float]): Function of which we want to find roots :math:`f(x)=0`.
        a (float): Lower bound of the interval.
        b (float): Upper bound of the interval.
        err (float): Tolerance of the result. It assures that the root is in :math:`[x-err, x+err]`. #TODO: Is this the interval?
        Nmax (int): Maximum number of iterations. Defaults to 100_000.
        x0 (float): Initial guess for the root. Defaults :math:`(a+b)/2`.

    Raises:
        ValueError: If, according to Bolzano's theorem, there cannot be roots in :math:`[a, b]`.
        ValueError: If the method, being at least one root in :math:`[a, b]`, fails to to compute the root.

    Returns:
        float: Root x such as f(x)=0 with a tolerance err.

    Examples:
        >>> f = lambda x: (x**3 - 5*x - 9)
        >>> secant(f, 2, 5, 1e-4)
        2.8551984513616424
    """
    if f(a)*f(b) >= 0:
        raise ValueError(f'f(a)*f(b) = {f(a)*f(b)} <0. \t No roots in this interval.')

    x_n = x0 if x0 is not None else (a+b)/2  # TODO: Check if there is a better initial guess
    x_previous = a - 1  # TODO: Check if there is a better initial guess
    f_x_n = f(x_n)

    for _ in range(1, Nmax+1):

        f_x_previous = f(x_previous)

        q_n = (f_x_n - f_x_previous) / (x_n - x_previous)
        x_previous = x_n

        x_n = x_n - f_x_n / q_n

        f_x_n = f(x_n)
        if abs(f_x_n) <= err:
            return x_n

    raise ValueError(f'Could not find a root in the interval [{a}, {b}] with tolerance {err} in {Nmax} iterations.')

### Newton's method

In [10]:
def newton(err: float = 1e-4, f: 'Callable[float]' = None, f_dev: 'Callable[float]' = None, x0: float = 0) -> float:
    r"""Newton's method to find roots of a function.

    Args:
        err (float): Desired error of the method.
        f (Callable[float], optional): Analytical function to find its roots. Its input is the point to be evaluated in. Defaults to None.
        f_dev (Callable[float], optional): Analytical derivative of the function. Its input is the point to be evaluated in. Defaults to None.
        x0 (float, optional): Initial guess of the root.
            Note that an inadequate first guess could lead to undesired outputs such as no roots or undesired roots.
            Defaults to 0.
        
    Returns:
        float|None: Root of the function or None if the algorithm reaches its recursion limit.
    """
    iter, iter_dict = 0, {0: x0} # Not necessary to store all the values, but anyway
    limit = sys.getrecursionlimit()

    while True:
        if iter + 10 >= limit:
            print(f'Iteration limit ({limit}) reached without finding any root. Try with other initial guess or changing the recursion limit. \
                    Maybe there are no roots.')
            return

        iter_dict[iter+1] = iter_dict[iter] - f(iter_dict[iter]) / f_dev(iter_dict[iter])

        if abs(iter_dict[iter+1] - iter_dict[iter]) < err:
            return iter_dict[iter+1]

        iter += 1

if __name__ == '__main__':

    def func(x):
        return (x+1)**2 - 1

    def func_dev(x):
        return 2*(x+1)

    newton(err=0.0001, f=func, f_dev=func_dev, x0=0.5)


## Exercise 5

In [ ]:
def newton_systems(f: 'Callable[float, ...]', J: 'Callable[float, ...]', vec0: np.ndarray, err: float) -> np.ndarray:
    r"""Solves systems of linear and nonlinear equations using the Newton method.

    Args:
        f (Callable[float, ...]): Vector function to find its roots.
        J (Callable[float, ...]): Jacobian of f.
        vec0 (np.ndarray): Initial guess of the solution. Avoid using guesses that make J a singular matrix (:math:`|J(vec0)| = 0`).
        err (float): Stopping criteria for the algorithm. Minimum difference between the to last consecutive solutions.

    Raises:
        ValueError: If the Jacobian of vec0 is a singular matrix, because its inverse cannot be computed.

    Returns:
        np.ndarray|None: Root of the function or None if the algorithm reaches its recursion limit.

    Examples:
        Solve
        ..math::
            `x^2+y^2-25=0  \\ x-y-2=0`
        With an adequate initial guess.

        >>> f = lambda x, y: [x**2 + y**2 -25,
                              x - y - 2]
        >>> J = lambda x, y: [[2*x, 2*y],
                              [1, -1]]
        >>> err = 1e-10
        >>> vec0 = [0, 0] #Invalid initial guess.
        >>> newton_systems(f, J, vec0, err)
        Raises numpy.linalg.LinAlgError: Singular matrix
        >>> vec0 = [1, 0] #Valid initial guess.
        >>> roots = newton_systems(f, J, vec0, err)
        >>> roots
        [4.39116499 2.39116499]
        >>> f(*roots)
        [-3.552713678800501e-15, -4.440892098500626e-16]
    """
    if np.linal.det(J(*vec0)) == 0:
        raise ValueError('Inverse of the Jacobian cannot be computed. It is a singular matrix (Determinant of the matrix is 0). ')

    iter, iter_dict = 0, {0: vec0}
    limit = sys.getrecursionlimit()

    while True:
        if iter + 10 >= limit:
            print(
                f'Iteration limit ({limit}) reached without finding any root. Try with other initial guess or changing the recursion limit.\
                     Maybe there are no roots.')
            return

        iter_dict[iter+1] = iter_dict[iter] - np.matmul(np.linalg.inv(J(*iter_dict[iter])), f(*iter_dict[iter]))

        if np.all(abs(iter_dict[iter + 1] - iter_dict[iter]) < err):
            return iter_dict[iter+1]

        iter += 1